In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
import attr
from twisted.internet import reactor
from txdbus import client
from txdbus.interface import DBusInterface, Method, Property, Signal
from txdbus.objects import DBusObject, DBusProperty

from pyxsession.twisted.util import returns_deferred
from pyxsession.dbus.marshmallow.schema import DBUS_FIELD
from pyxsession.dbus.marshmallow.fields import (
    DBusField, Int32, UInt32, Str, List, Tuple
)
from pyxsession.dbus.service import Service

      
@attr.s
class Payload:
    message = attr.ib(default='', metadata={DBUS_FIELD: Str()})



In [3]:
test_service = Service('org.jfhbrook.testing123')

echo_obj = test_service.obj('/Echo')

@echo_obj.method([Payload], Str())
def echo(payload):
    return f'You said: {payload.message}'

In [4]:
conn = await client.connect(reactor)

In [5]:
server = await test_service.server(conn)

server

Server(connection=<txdbus.client.DBusClientConnection object at 0x7f60bd6aef28>, service=<pyxsession.dbus.service.Service object at 0x7f60bd6ba198>, bus_name=[1], dbus_obj_cls=<class 'pyxsession.dbus.server.Echo'>, dbus_obj=<pyxsession.dbus.server.Echo object at 0x7f60bd5e0be0>)

In [6]:
client = await test_service.client(conn)

client

Client(service=<pyxsession.dbus.service.Service object at 0x7f60bd6ba198>, remote_objs={'/Echo': <txdbus.objects.RemoteDBusObject object at 0x7f60bd5fbac8>})

In [8]:
await client.Echo.echo(Payload('hello'))

'You said: hello'